In [ ]:
!pip install ipyfilechooser
!pip install scikit-image
import os
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import widgets
from ipyfilechooser import FileChooser
 
# Helper function to calculate similarity
def calculate_similarity(image1_path, image2_path):
    image1 = Image.open(image1_path).convert('L')
    image2 = Image.open(image2_path).convert('L')

    image1 = image1.resize((256, 256))
    image2 = image2.resize((256, 256))

    image1_np = np.array(image1)
    image2_np = np.array(image2)

    similarity_index, _ = ssim(image1_np, image2_np, full=True)
    return similarity_index

# Widget to upload an image
file_chooser = FileChooser('/lakehouse/default/Files/upload_files') 
display(file_chooser)

def find_sole_images(root_dir):
    sole_images = []
    valid_extensions = {'.png', '.jpg', '.jpeg', '.jfif', '.bmp', '.gif', '.tiff'}
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_name, file_extension = os.path.splitext(file)
            if file_name.lower() == 'sole' and file_extension.lower() in valid_extensions:
                sole_images.append(os.path.join(subdir, file))
    return sole_images

def compare_uploaded_image(file_chooser, root_dir):

    uploaded_image_path = file_chooser.selected

    if uploaded_image_path:
        print(f"Selected file: {uploaded_image_path}")
        
        
        sole_images = find_sole_images(root_dir)

        similarities = []
        
        for sole_image in sole_images:
            similarity = calculate_similarity(uploaded_image_path, sole_image)
            # Add the similarity score and the sole image path to the results  
            similarities.append((os.path.relpath(sole_image, root_dir), similarity))
        
        # Sort the results by similarity in descending order - the most likely
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Create a DataFrame to store the results
        df = pd.DataFrame(similarities, columns=['File Path', 'Similarity'])
        
        # Save the results to a CSV file
        csv_path = '/lakehouse/default/Files/data/similarity_results.csv'
        df.to_csv(csv_path, index=False)
        
        # Display the results as a table
        display(HTML(df.to_html(index=False)))
        
        print(f'Results saved to {csv_path}')
    else:
        print("Please upload an image first.")

root_dir = '/lakehouse/default/Files/Shoe'  # Need to check this is the right file path 

compare_button = widgets.Button(description="Compare Images")
display(compare_button)

def on_compare_button_clicked(b):
    compare_uploaded_image(file_chooser, root_dir)

compare_button.on_click(on_compare_button_clicked)


In [ ]:
!pip install tensorflow ipyfilechooser scikit-image

import os
import numpy as np
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import widgets
from ipyfilechooser import FileChooser
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

# Load the ResNet50 model
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Helper function to calculate similarity using CNN feature extraction
def calculate_similarity(image1_path, image2_path):
    def extract_features(img_path):
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        features = model.predict(img_data)
        return features

    features1 = extract_features(image1_path)
    features2 = extract_features(image2_path)

    similarity = cosine_similarity(features1, features2)[0][0]
    return similarity

# Widget to upload an image
file_chooser = FileChooser('/lakehouse/default/Files/upload_files')
display(file_chooser)

def find_sole_images(root_dir):
    sole_images = []
    valid_extensions = {'.png', '.jpg', '.jpeg', '.jfif', '.bmp', '.gif', '.tiff'}
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_name, file_extension = os.path.splitext(file)
            if file_name.lower() == 'sole' and file_extension.lower() in valid_extensions:
                sole_images.append(os.path.join(subdir, file))
    return sole_images

def compare_uploaded_image(file_chooser, root_dir):
    uploaded_image_path = file_chooser.selected

    if uploaded_image_path:
        print(f"Selected file: {uploaded_image_path}")
        
        sole_images = find_sole_images(root_dir)
        similarities = []
        
        for sole_image in sole_images:
            similarity = calculate_similarity(uploaded_image_path, sole_image)
            # Add the similarity score and the sole image path to the results  
            similarities.append((os.path.relpath(sole_image, root_dir), similarity))
        
        # Sort the results by similarity in descending order - the most likely
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Create a DataFrame to store the results
        df = pd.DataFrame(similarities, columns=['File Path', 'Similarity'])
        
        # Save the results to a CSV file
        csv_path = '/lakehouse/default/Files/data/similarity_results.csv'
        df.to_csv(csv_path, index=False)
        
        # Display the results as a table
        display(HTML(df.to_html(index=False)))
        
        print(f'Results saved to {csv_path}')
    else:
        print("Please upload an image first.")

root_dir = '/lakehouse/default/Files/Shoe'  # Ensure this is the right file path 

compare_button = widgets.Button(description="Compare Images")
display(compare_button)

def on_compare_button_clicked(b):
    compare_uploaded_image(file_chooser, root_dir)

compare_button.on_click(on_compare_button_clicked)
